In [1]:
import torch
import pickle
import regex as re


import os
import random as rand
from tqdm import tqdm, trange

import numpy as np
import pandas as pd
# from clean_text import stopwords_make, punctstr_make, unicode_make, apache_tokenize, clean_sentence_apache 

import matplotlib.pyplot as plt


from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import transformers
import sys, argparse
import numpy as np
import re
import random
from tqdm import tqdm
from collections import Counter
from transformers import BertTokenizer, BertModel
from collections import Counter

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from transformers import AdamW
from transformers import get_scheduler


Bad key savefig.frameon in file C:\Users\theth\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\theth\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\theth\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably need to get an updated matplotlibrc fil

In [2]:
def open_test_data(path):
    return open(path, 'rb')
with open_test_data('/home/jovyan/work/tlu_storage/training_cultural_preprocessed_100321.pkl') as f:
    cult = pickle.load(f)

# with open_test_data('/home/jovyan/work/tlu_storage/training_demographic_preprocessed_100321.pkl') as f:
#     demog = pickle.load(f)

# with open_test_data('/home/jovyan/work/tlu_storage/training_orgs_preprocessed_100321.pkl') as f:
#     orgs = pickle.load(f)

# with open_test_data('/home/jovyan/work/tlu_storage/training_relational_preprocessed_100321.pkl') as f:
#     rela = pickle.load(f)
    


In [3]:
import itertools
full_text = []

for i in cult['text']:
    joined = list(itertools.chain(*i))
    full_text.append(" ".join(joined))


# ' '.join(cult['text'][0][0])

# full_text_demog = []
# for i in demog['text']:
#     joined = list(itertools.chain(*i))
#     full_text_demog.append(" ".join(joined))


# full_text_orgs = []
# for j in orgs['text']:
#     joined = list(itertools.chain(*j))
#     full_text_orgs.append(" ".join(joined))


# full_text_rela = []
# for j in rela['text']:
#     joined = list(itertools.chain(*j))
#     full_text_rela.append(" ".join(joined))

In [4]:
cult['full_text'] = full_text
# demog['full_text'] = full_text_demog
# orgs['full_text'] = full_text_orgs
# rela['full_text'] = full_text_rela

In [5]:
def remove_tags(article):
    article = re.sub('<plain_text> <page sequence="1">', '', article)
    article = re.sub(r'</page>(\<.*?\>)', ' \n ', article)
    # xml tags
    article = re.sub(r'<.*?>', '', article)
    article = re.sub(r'<body.*\n\s*.*\s*.*>', '', article)
    return article

tags_removed = [remove_tags(art) for art in cult['full_text']]
# tags_removed_demog = [remove_tags(art) for art in demog['full_text']]
# tags_removed_org = [remove_tags(art) for art in orgs['full_text']]
# tags_removed_rela = [remove_tags(art) for art in rela['full_text']]
cult['text_no_tags'] = tags_removed
# demog['text_no_tags'] = tags_removed_demog
# orgs['text_no_tags'] = tags_removed_org
# rela['text_no_tags'] = tags_removed_rela

In [6]:
## remove cult_label that are 0.5

cult = cult[cult['cultural_score']!=0.5]
cult

,text,cultural_score,primary_subject,edited_filename,article_name,full_text,text_no_tags
0,"[[Where, Do, Interorganizational, Networks, Co...",0.0,Sociology,10.1086_210179,Where Do Interorganizational Networks Come From?,Where Do Interorganizational Networks Come Fro...,Where Do Interorganizational Networks Come Fro...
1,"[[Civil, Rights, Law, at, Work:, Sex, Discrimi...",1.0,Sociology,10.1086_210317,Civil Rights Law at Work: Sex Discrimination a...,Civil Rights Law at Work: Sex Discrimination a...,Civil Rights Law at Work: Sex Discrimination a...
2,"[[Between, Markets, and, Politics:, Organizati...",0.0,Sociology,10.1086_231084,Between Markets and Politics: Organizational R...,Between Markets and Politics: Organizational R...,Between Markets and Politics: Organizational R...
3,"[[World, Society, and, the, Nation-State, John...",1.0,Sociology,10.1086_231174,World Society and the Nation‐State,World Society and the Nation-State John Meyer ...,World Society and the Nation-State John Meyer ...
4,"[[<body, xmlns:xlink=""http://www..org//xlink""]...",1.0,Sociology,10.1086_382347,Kinship Networks and Entrepreneurs in China’s ...,"<body xmlns:xlink=""http://www..org//xlink"" xml...",Introduction Economists have long concurred t...
...,...,...,...,...,...,...,...
914,"[[Institutionalized, Organizations:, Formal, S...",1.0,Sociology,10.2307_2778293,Institutionalized Organizations: Formal Struct...,Institutionalized Organizations: Formal Struct...,Institutionalized Organizations: Formal Struct...
915,"[[The, Social, Construction, of, Organizationa...",1.0,Management & Organizational Behavior,10.2307_2667051,The Social Construction of Organizational Know...,The Social Construction of Organizational Know...,The Social Construction of Organizational Know...
916,"[[Organizational, Structure, and, the, Institu...",1.0,Management & Organizational Behavior,10.2307_2392303,Organizational Structure and the Institutional...,Organizational Structure and the Institutional...,Organizational Structure and the Institutional...
917,"[[Institutional, Sources, of, Change, in, the,...",1.0,Management & Organizational Behavior,10.2307_2392383,Institutional Sources of Change in the Formal ...,Institutional Sources of Change in the Formal ...,Institutional Sources of Change in the Formal ...


In [7]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1070 with Max-Q Design


In [8]:
class BERTClassifier(nn.Module):

        
    def __init__(self, params):
        super().__init__()
        
        if 'dropout' in params:
            self.dropout = nn.Dropout(p=params['dropout'])
        else:
            self.dropout = None
            
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False, do_basic_tokenize=False)
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        self.num_labels = params["label_length"]

        self.fc = nn.Linear(768, self.num_labels)

    def get_batches(self, all_x, all_y, batch_size=10):

        """ Get batches for input x, y data, with data tokenized according to the BERT tokenizer 
        (and limited to a maximum number of WordPiece tokens """

        batches_x=[]
        batches_y=[]

        for i in range(0, len(all_x), batch_size):

            current_batch=[]

            x=all_x[i:i+batch_size]

            batch_x = self.tokenizer(x, padding=True, truncation=True, return_tensors="pt")
            batch_y=all_y[i:i+batch_size]

            batches_x.append(batch_x.to(device))
            batches_y.append(torch.LongTensor(batch_y).to(device))

        return batches_x, batches_y


    def forward(self, batch_x): 

        bert_output = self.bert(input_ids=batch_x["input_ids"],
                         attention_mask=batch_x["attention_mask"],
                         token_type_ids=batch_x["token_type_ids"],
                         output_hidden_states=True)

        # We're going to represent an entire document just by its [CLS] embedding (at position 0)
        # And use the *last* layer output (layer -1)
        # as a result of this choice, this embedding will be optimized for this purpose during the training process.

        bert_hidden_states = bert_output['hidden_states']

        out = bert_hidden_states[-1][:,0,:]
        
        if self.dropout:
            out = self.dropout(out)

        out = self.fc(out)

        return out.squeeze()

    def evaluate(self, batch_x, batch_y):

        self.eval()
        corr = 0.
        total = 0.

        with torch.no_grad():
            for x, y in zip(batch_x, batch_y):
                y_preds = self.forward(x)
                for idx, y_pred in enumerate(y_preds):
                    prediction=torch.argmax(y_pred)
                    if prediction == y[idx]:
                        corr += 1.
                    total+=1                          
        return corr/total



In [9]:
def oversample_shuffle(X, y, random_state=52):
    """
    Oversamples X and y for equal class proportions
    """
    
    fakeX = np.arange(len(X), dtype=int).reshape((-1, 1))
    ros = RandomOverSampler(random_state=random_state, sampling_strategy=1.0)
    fakeX, y = ros.fit_resample(fakeX, y)
    p = np.random.permutation(len(fakeX))
    fakeX, y = fakeX[p], y[p]
    X = X.iloc[fakeX.reshape((-1, ))]
    return X.to_numpy(), y.to_numpy()

def create_datasets(texts, scores, test_size=0.2, random_state=42):
    """
    A function to split the texts (X variables) and the scores (y variables), and oversample all at once
    """
    
    X_train1, X_test1, y_train1, y_test1 = train_test_split(
        texts, scores, test_size=test_size, random_state=random_state)
    
    X_train1, y_train1 = oversample_shuffle(X_train1, y_train1, random_state=random_state)
    X_test1, y_test1 = oversample_shuffle(X_test1, y_test1, random_state=random_state)
    
    return X_train1, y_train1, X_test1, y_test1
    
def train_bert(texts, scores, params, test_size=0.2, random_state=42, save=False):
    """
    Trains an entire BERT model with the given parameters, then returns the best accuracy out of all epochs
    
    Parameters list: a dict with the following keys (all optional)
    
    {
    batch_size: The batch size as far as the optimizer is concerned. default 32
    max_memory_size: The largest batch to actually use. This is because large batches wont fit on many GPUs.
                        Gradient accumulation will then let us run optimization on batch_size instead. default 4
    
    num_epochs: The total number of epochs to train. default 8
    lr: The learning rate to use for the final linear layer. default 1e-3
    pretrained_lr = A separate learning rate for only the pretrained model. default 1e-5
    betas = The betas parameter in the Adam optimizer. default (0.9, 0.98)
    eps = The eps parameter in the Adam optimizer. default 1e-9
    weight_decay = The weight_decay parameter in the Adam optimizer. default 0.005
    num_warmup_steps = The number of warmup steps (each step is a batch). default 0
    num_training_steps = The number of training steps in the algorithm. default num_epochs * len(batch_x) - num_warmup_steps
    
    dropout: A float (i.e. 0.1) if dropout is desired. default None
    }
    """
    np.random.seed(random_state)
    torch.random.manual_seed(random_state)
    
    texts = texts.copy()
    scores = scores.copy()
    
    bert_model = BERTClassifier(params={**params, "label_length": 2})
    bert_model.to(device)
    
    
    X_train1, y_train1, X_test1, y_test1 = create_datasets(
        texts, scores, test_size=test_size, random_state=random_state)

    batch_size = params['batch_size'] if 'batch_size' in params else 32
    max_memory_size = params['max_memory_size'] if 'max_memory_size' in params else 4
    
    batch_x, batch_y = bert_model.get_batches(list(X_train1), list(y_train1), batch_size=max_memory_size)
    dev_batch_x, dev_batch_y = bert_model.get_batches(list(X_test1),list( y_test1), batch_size=max_memory_size)

    
    num_epochs = params['num_epochs'] if 'num_epochs' in params else 8
    lr = params['lr'] if 'lr' in params else 1e-3
    pretrained_lr = params['pretrained_lr'] if 'pretrained_lr' in params else 1e-5
    betas = params['betas'] if 'betas' in params else (0.9, 0.98)
    eps = params['eps'] if 'eps' in params else 1e-9
    weight_decay = params['weight_decay'] if 'weight_decay' in params else 0.005
    num_warmup_steps = params['num_warmup_steps'] if 'num_warmup_steps' in params else 0
    num_training_steps = params['num_training_steps'] if 'num_training_steps' in params \
        else num_epochs * len(batch_x) - num_warmup_steps
    
    
    trainable_parameters = list(
        params for params in bert_model.parameters() if params.requires_grad)
    pretrained_params = set(trainable_parameters) & set(bert_model.bert.parameters())
    new_params = set(trainable_parameters) - pretrained_params
    grouped_trainable_parameters = [
        {
            'params': list(pretrained_params),
            'lr': pretrained_lr,
        },
        {
            'params': list(new_params),
            'lr': lr,
        },
    ]
    
    #torch.optim.
    optimizer = AdamW(
        grouped_trainable_parameters, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
    
    scheduler = get_scheduler(
        'linear',
        optimizer=optimizer, 
        num_warmup_steps=num_warmup_steps, 
        num_training_steps=num_training_steps
    )

    cross_entropy=nn.CrossEntropyLoss()

    best_dev_acc = 0.

    for epoch in tqdm(range(num_epochs)):
        bert_model.train()

        # Train
        optimizer.zero_grad()
        for i, (x, y) in enumerate(zip(batch_x, batch_y)):
            y_pred = bert_model.forward(x)
            loss = cross_entropy(y_pred.view(-1, bert_model.num_labels), y.view(-1))
            loss.backward()
            if (i + 1) % (batch_size // max_memory_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

        # Evaluate
        dev_accuracy=bert_model.evaluate(dev_batch_x, dev_batch_y)
        if epoch % 1 == 0:
#             print("Epoch %s, dev accuracy: %.3f" % (epoch, dev_accuracy))
            if dev_accuracy > best_dev_acc:
                if save:
                    torch.save(bert_model.state_dict(), save)
                best_dev_acc = dev_accuracy
    if save:
        bert_model.load_state_dict(torch.load('best-model-parameters.pt'))
        
    print("\nBest Performing Model achieves dev accuracy of : %.3f" % (best_dev_acc))
    return best_dev_acc

In [10]:
def cross_validate(texts, scores, params, folds=5, seed=42):
    np.random.seed(seed)
    states = np.random.randint(0, 100000, size=folds)
    accs = []
    for state in states:
        print(state)
        accs.append(train_bert(texts, scores, params, test_size=round(1/folds, 2), random_state=state, save=False))
    return np.mean(accs)

In [11]:
# # 0.799367986285495
# cross_validate(cult['text_no_tags'], cult['cultural_score'], dict())

In [12]:
#0.8164062683262356
# cross_validate(cult['text_no_tags'], cult['cultural_score'], {'batch_size': 16, 'lr': 1e-4})

In [13]:
#0.820989455049683
# cross_validate(cult['text_no_tags'], cult['cultural_score'], {'batch_size': 16, 'lr': 1e-4, 'dropout':0.1})

15795


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [15:12<00:00, 114.02s/it]



Best Performing Model achieves dev accuracy of : 0.784
860


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [22:53<00:00, 171.66s/it]



Best Performing Model achieves dev accuracy of : 0.870
76820


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [28:46<00:00, 215.82s/it]



Best Performing Model achieves dev accuracy of : 0.822
6265


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [29:00<00:00, 217.61s/it]


Best Performing Model achieves dev accuracy of : 0.830


0.820989455049683

In [14]:
# 0.8206066141611448
# cross_validate(cult['text_no_tags'], cult['cultural_score'], {'batch_size': 16, 'lr': 1e-4, 'num_warmup_steps':40})

15795


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [28:33<00:00, 214.16s/it]



Best Performing Model achieves dev accuracy of : 0.779
860


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [28:30<00:00, 213.87s/it]



Best Performing Model achieves dev accuracy of : 0.865
76820


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [29:07<00:00, 218.40s/it]



Best Performing Model achieves dev accuracy of : 0.839
6265


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [29:13<00:00, 219.14s/it]


Best Performing Model achieves dev accuracy of : 0.795


0.8206066141611448

In [15]:
# 0.8322880870561283

# cross_validate(cult['text_no_tags'], cult['cultural_score'], {'batch_size': 16, 'lr': 1e-4, 'num_warmup_steps':40, 'dropout':0.1})

15795


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [28:46<00:00, 215.87s/it]



Best Performing Model achieves dev accuracy of : 0.800
860


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [28:45<00:00, 215.71s/it]



Best Performing Model achieves dev accuracy of : 0.875
76820


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [30:51<00:00, 231.50s/it]



Best Performing Model achieves dev accuracy of : 0.839
6265


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [32:06<00:00, 240.83s/it]


Best Performing Model achieves dev accuracy of : 0.812


0.8322880870561283

In [16]:
#0.8120893881846133

# cross_validate(cult['text_no_tags'], cult['cultural_score'], {'batch_size': 32, 'lr': 1e-4, 'dropout':0.1})

15795


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [31:54<00:00, 239.36s/it]



Best Performing Model achieves dev accuracy of : 0.774
860


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [31:26<00:00, 235.86s/it]



Best Performing Model achieves dev accuracy of : 0.840
76820


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [33:21<00:00, 250.20s/it]



Best Performing Model achieves dev accuracy of : 0.817
6265


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [32:10<00:00, 241.33s/it]


Best Performing Model achieves dev accuracy of : 0.847


0.8120893881846133